In [1]:
import duckdb
import xml.etree.ElementTree as ET
# Import classifier
from ipynb.fs.full.group_classifier import predict_batch
print("Import successful!!")

2025-06-04 13:45:15.715750: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 13:45:15.729030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749037515.743797  394960 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749037515.748361  394960 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 13:45:15.764732: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Import successful
Import successful!!


In [2]:
 # Connect to sql database
con = duckdb.connect(database='../data/database/german-parliament', read_only=False)

In [3]:
def preprocess_speech(speech:tuple) -> tuple[int, list[str]]:
    # We only need the speech it to later identify the origin speech
    speech_id = test_speech[0]
    text = test_speech[5]
    # Parse xml from string will throw ParseError if not parseable
    root = ET.fromstring(text)
    # We only need paragraphs -> this filters unecessary information, such as comments or applause
    paragraphs = root.findall('p')
    paragraphs_text = []
    # Get text of paragraphs
    for p in paragraphs:
        # Use this to filter out speaker information
        if p.attrib.get("klasse") == "redner":
            continue
        else:
            p_text = p.text.strip()
            item = {"paragraphs":p_text}
            paragraphs_text.append(item) #get text and remove potential irrelevant whitespaces

    return (speech_id, paragraphs_text)
    

In [99]:
test_speech = con.sql("select * from speech where id like '%ID%' order by random() limit 1 ").fetchone()

In [100]:
speech_id, paragraphs = preprocess_speech(test_speech)

print(speech_id)
print(paragraphs)

ID1913007600
[{'paragraphs': 'Herzlichen Dank.\xa0– Herr Präsident! Liebe Kolleginnen und Kollegen! Der Haushalt des Entwicklungsministeriums ist typischerweise auch zu bezeichnen als der Haushalt der Nachhaltigkeit, der Haushalt für den Weltzukunftsvertrag, für die globalen Nachhaltigkeitsziele.'}, {'paragraphs': 'Damit ein solcher Haushalt ausgestattet werden kann, muss aber auch das Land selbst, muss Deutschland nachhaltig sein. Das bedeutet, auch seine eigenen Finanzen nachhaltig auszugestalten, und zwar im Interesse nachfolgender Generationen. Deswegen ist es großartig, dass wir hier erneut ohne neue Schulden auskommen. Dafür auch herzlichen Dank an den Bundesfinanzminister!'}, {'paragraphs': 'Das sehen nicht alle Fraktionen so. Ich glaube, die Grünen haben jetzt im Zusammenhang mit ihren eigenen Vorstellungen zum Klimaschutz vor, neue Schulden aufzunehmen in der Größenordnung von 20\xa0Milliarden Euro. Das ist nicht unser Weg, weil das nämlich auch bedeuten würde, dass wir eben n

In [101]:
classified_paragraphs = predict_batch(paragraphs)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Batch size: 8


In [102]:
# Maybe adjust this set to tokenizer's special tokens
SPECIAL = {"[CLS]", "[SEP]", "[PAD]", "[UNK]"}
for index, p in enumerate(classified_paragraphs):
    print(f"Processing paragraph {index}")
    groups = []
    group_tmp = []
    group_started = False
    entity = "" # This hold the current entity. e.g. EOPOL for B-EOPOL
    for token, label in p:
        # If token is a special token like [CLS] skip it
        if token in SPECIAL:
            continue
        # Check for begin of group
        elif label.startswith("B-"):
            group_started = True
            entity = label[2:]
            if group_tmp:
                groups.append(group_tmp)
                group_tmp = [] # New Group will begin
            # Append new beginning label
            group_tmp.append((token, label))
        # It is checked that 1) we have an inside label and 2) There was a B- label before!
        elif label.startswith("I-") and group_started:  
            # Then we check if the entity matches
            if label[2:] != entity:
                print(f"Current label: {label[2:]} doesn't match beginning label: {entity}")
                # Break current group because of the miss-label
                group_started = False
            else:
            # If all tests hold, we append the token and its label to the current group
                group_tmp.append((token, label))
        elif label == 'O':
            # An 'O' Tag is always outside. Thus if we scan one, it means that the current group is over
            if group_tmp:
                groups.append(group_tmp)
                group_tmp = [] # New Group will begin
            group_started = False
        else:
            print(f"Filtered faulty classification: ({token}, {label})")
    
    # Flush last word
    if group_tmp:
        groups.append(group_tmp)
    
    for group in groups:
        _, entity = group[0]
        entity = entity[2:] # Get rid of  "B-" tag
        tokens = [item[0] for item in group]
        group_joined = smart_join(tokens)
        print(f"\tGroup: {entity}, Text: {group_joined}")

    print("\n")


Processing paragraph 0
	Group: EPPOL, Text: Herr Präsident
	Group: EOPOL, Text: des Entwicklungsministeriums


Processing paragraph 1
	Group: GPE, Text: Deutschland
	Group: EPPOL, Text: den Bundesfinanzminister


Processing paragraph 2
	Group: EOPOL, Text: alle Fraktionen
	Group: EOPOL, Text: die Grünen
	Group: EPPOL, Text: Bundesminister Müller


Processing paragraph 3
	Group: EOPOL, Text: das Auswärtige Amt


Processing paragraph 4
	Group: EPPOL, Text: Lieber Bundesminister
	Group: EPPOL, Text: lieber Gerd
	Group: EOPOL, Text: des Entwicklungsministeriums
	Group: PFUNK, Text: viele Menschen, die sich da ehrenamtlich oder hauptamtlich engagieren


Processing paragraph 5
Filtered faulty classification: (E, I-EOPOL)
	Group: EOPOL, Text: den Europäischen Entwicklungsfonds
	Group: EOPOL, Text: der Mitgliedstaaten
	Group: PNAT, Text: die Osteuropäer
	Group: EOPOL, Text: der Europäischen Union
	Group: EPPOL, Text: Kollege von Holtz
	Group: EOPOL, Text: der EU
	Group: EOPOL, Text: der
	Group

Group: PFUNK, Text: der Arbeitnehmerinnen und Arbeitnehmer
Group: PFUNK, Text: der Arbeitnehmerinnen und Arbeitnehmer
Group: EOPOL, Text: das Bündnis für Arbeit
Group: PFUNK, Text: der in den Betrieben Beschäftigten


In [55]:
def smart_join(tokens):
    """
    Joins a list of word tokens into a single string, handling punctuation
    and sub-word prefixes ('##') correctly.

    Args: @todo
    """
    result = []
    # Define punctuation that should not have a preceding space
    no_space_before = {',', '.', '?', '!', ';', ':', ')'}
    
    for i, token in enumerate(tokens):
        # If it's the very first token, a punctuation mark, or a sub-word,
        # don't add a leading space.
        if i > 0 and token not in no_space_before and not token.startswith('##'):
            result.append(' ')
            
        # Append the token itself, removing any '##' prefixes
        result.append(token.replace('##', ''))
        
    return "".join(result)